# Sleep stage classification on Sleep-edf data set

In this project, we will use deep neural network on raw EEG data for sleep stage classification.

This work is based on the MNE-Python and the mne-torch repository, as well as

Chambon, S., Galtier, M. N., Arnal, P. J., Wainrib, G., & Gramfort, A. (2018). A deep learning architecture for temporal sleep stage classification using multivariate and multimodal time series. IEEE Transactions on Neural Systems and Rehabilitation Engineering, 26(4), 758-769.

# Sleep staging

The method of determining a person's sleep stage by studying their EEG and other physiological signs is known as sleep staging. Sleep recordings are generally separated into 30-second windows, with each window assigned to one of five groups ("stages"):